In [ ]:
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

In [ ]:
# Загружаем один из файлов
data = np.load('../data/processed//Unit2_win50_str1_smp10.npz')

# Смотрим, какие массивы внутри
print(list(data.keys()))

['sample', 'label']


In [9]:
# Смотрим на форму массивов
x_data = data['sample']
y_data = data['label']

print('Форма X (входные данные):', x_data.shape)
print('Форма Y (целевые значения):', y_data.shape)

Форма X (входные данные): (50, 20, 85266)
Форма Y (целевые значения): (85266,)


In [ ]:
def load_and_merge_data(npz_units):
    sample_array_lst = []
    label_array_lst = []
    for npz_unit in npz_units:
      loaded = np.load(npz_unit)
      sample_array_lst.append(loaded['sample'])
      label_array_lst.append(loaded['label'])
    sample_array = np.dstack(sample_array_lst)
    label_array = np.concatenate(label_array_lst)
    sample_array = sample_array.transpose(2, 0, 1)
    return sample_array, label_array

processed_dir = '../data/processed/'

# Собираем пути к файлам для train и test
train_files = [os.path.join(processed_dir, f) for f in os.listdir(processed_dir) if f.startswith(('Unit2_', 'Unit5_', 'Unit10_', 'Unit16_', 'Unit18_', 'Unit20_'))]
test_files = [os.path.join(processed_dir, f) for f in os.listdir(processed_dir) if f.startswith(('Unit11_', 'Unit14_', 'Unit15_'))]
print(train_files)

# Загружаем данные
X_train, y_train = load_and_merge_data(train_files)
X_test, y_test = load_and_merge_data(test_files)

print('Размер обучающей выборки (X):', X_train.shape)
print('Размер обучающей выборки (y):', y_train.shape)
print('Размер тестовой выборки (X):', X_test.shape)
print('Размер тестовой выборки (y):', y_test.shape)

['../data/processed/samples_v1/Unit16_win50_str1_smp10.npz', '../data/processed/samples_v1/Unit5_win50_str1_smp10.npz', '../data/processed/samples_v1/Unit18_win50_str1_smp10.npz', '../data/processed/samples_v1/Unit20_win50_str1_smp10.npz', '../data/processed/samples_v1/Unit2_win50_str1_smp10.npz', '../data/processed/samples_v1/Unit10_win50_str1_smp10.npz']
Размер обучающей выборки (X): (526051, 50, 20)
Размер обучающей выборки (y): (526051,)
Размер тестовой выборки (X): (125227, 50, 20)
Размер тестовой выборки (y): (125227,)


In [ ]:
# Определяем форму входных данных из X_train
n_timesteps, n_features = X_train.shape[1], X_train.shape[2]

# --- Создаем простую LSTM модель ---
model = Sequential()
model.add(LSTM(64, input_shape=(n_timesteps, n_features), return_sequences=True)) # return_sequences=True, если следующий слой тоже LSTM
model.add(Dropout(0.2))
model.add(LSTM(32))
model.add(Dropout(0.2))
model.add(Dense(1)) # Один выход, так как предсказываем одно число - RUL

# Компилируем модель
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

print(model.summary())

# --- Обучаем модель ---
history = model.fit(X_train, y_train, 
                    epochs=2, 
                    batch_size=512, 
                    validation_split=0.2, # Используем часть данных для валидации на лету
                    verbose=1)

# --- Оцениваем на тестовых данных ---
loss, mae = model.evaluate(X_test, y_test, verbose=0)
print(f'\nTest MAE: {mae:.2f}')

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_3 (LSTM)                   │ (None, 50, 64)         │        21,760 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 50, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_4 (LSTM)                   │ (None, 32)             │        12,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 34,209 (133.63 KB)

 Trainable params: 34,209 (133.63 KB)

 Non-trainable params: 0 (0.00 B)

None
822/822 ━━━━━━━━━━━━━━━━━━━━ 147s 178ms/step - loss: 853.2334 - mae: 23.5538 - val_loss: 680.1964 - val_mae: 22.2617

Test MAE: 16.43
